# Búsqueda semántica
Autor: Eric S. Tellez <eric.tellez@infotec.mx>

Cuando la información esta poco específicada, las palabras adecuadas podrían ser dificiles de tener o limitar. En estos casos, la representaciones semánticas que permiten buscar _lo que se desea_ por medio de conceptos nos acerca más a la posibilidad de obtener información útil.

Como anteriormente se presento, una de las representaciones semánticas más aceptadas son aquellas basadas en word embeddings. Recordando, estas son representaciones vectoriales de cada palabra, donde la semántica se asocia con la estructura en un espacio métrico, i.e., cercano en la métrica significa similar semánticamente.

# Representación semántica de documentos basada en _word embeddings_
Un documento puede verse como una bolsa de palabras, como en el modelado tradicional, pero en lugar de usar los términos como símbolos se pueden usar los vectores semánticos. El como usar esta nube de puntos multidimensional para obtener una representación computacionalmente manejable y a la vez eficaz, es un tema que ha llevado al desarrollo de modelos cada vez más complejos, como pueden ser los grandes modelos de lenguaje. 

La manera forma más directa de definir una representación semántica de documentos es el uso de centroides, esto es, un vector de la misma dimensión que sumariza a un conjunto de vectores usando su media geométrica. De manera más precisa, sea $E$ una matrix $m \times n$ de embeddings del vocabulario, i.e., considerando un vocabulario de tamaño $n$ y dimensión $m$.
Sea $D_{m,\ell}$ la submatriz de $E$ que contiene los $\ell$ vectores del documento $\textsf{doc}$. El prototipo $\vec{d}$ de $D$ esta definido de la siguiente forma:

\begin{equation}
\vec{d} = \frac{1}{\ell} \sum_{i=1}^\ell D_i
\end{equation}


Donde $D_i$ es el i-ésimo vector columna de $D$. Dado que se usa el coseno como similitud, también es factible el cálculo como la suma vectorial normalizada, esto es,

$$ \vec{d} = \frac{\sum_{i=1}^\ell D_i}{\lVert \sum_{i=1}^\ell D_i \rVert}$$

También es posible añadir información local basada en la representación de bolsa de plabras. Por ejemplo, realizar una suma pesada usando el peso de las palabras mediante la frecuencia normalizada de término (TF) o la probabilidad de término $pt$, ver unidad 3 para más información.

$$\vec{d} = \sum_{i=1}^{\ell} \textsf{TF}(t_i, \textsf{doc}) \cdot D_i$$

Donde $t_i$ es el $i$-ésimo término de $\textsf{doc}$. Note que el orden no se captura y solo es necesario para la notación.

# Ejemplo


In [1]:
using Pkg
Pkg.activate(".")

using SimilaritySearch, TextSearch, Plots, KNearestCenters, LinearAlgebra, Embeddings, HypertextLiteral, CodecZlib, JSON, Base64, Random
using Downloads: download

  Activating project at `~/IR-2022/Unidades`


In [2]:
include("knn.jl")

scores (generic function with 1 method)

In [3]:
struct ValidVocabulary{DictType} <: AbstractTokenTransformation
    valid::DictType
end

TextSearch.transform_unigram(S::ValidVocabulary, tok) = haskey(S.valid, tok) ? tok : nothing

function text_model_and_vectors(
        corpus, vocab;
        localweighting=TpWeighting(),
        globalweighting=BinaryGlobalWeighting(),
        nlist=[1],
        qlist=[],
        slist=[],
        group_usr=false,
        group_url=true,
        group_num=true,
        del_diac=true,
        lc=true
    )

    voc = Vocabulary(length(corpus))
    for v in vocab
       push!(voc, v)
    end
    
    tt = ValidVocabulary(voc.token2id)
    textconfig = TextConfig(; group_usr, group_url, del_diac, lc, group_num, nlist, qlist, slist, tt)
    tokenize_and_append!(voc, textconfig, corpus)
    model = VectorModel(globalweighting, localweighting, voc)
    (; textconfig, model, voc)
end


text_model_and_vectors (generic function with 1 method)

In [4]:
function embeddings(embname="MX.vec")
    embfile = "../data/$embname"
    !isfile(embfile) && download("http://geo.ingeotec.mx/~sadit/regional-spanish-models/$embname", embfile)
    emb = load_embeddings(FastText_Text, embfile)  # you can change with any of the available embeddings in `Embeddings`
    for c in eachcol(emb.embeddings)
        normalize!(c)
    end
    
    (; X=emb.embeddings, vocab=emb.vocab)
end

function vectorize_as_prototype!(c, text, E, T)
    x = vectorize(T.model, T.textconfig, text; normalize=false)
    if length(x) == 1 && haskey(x, 0)
        #@warn "empty vector $(Int(i)) selecting a random vector for it " # $(corpus[i])
        rand!(c)
    else
        for (id, weight) in x
            id == 0 && continue
             c .= c .+ weight .* view(E.X, :, id)
        end
    end
    
    normalize!(c)
end

function vectorize_corpus_as_prototypes(corpus, E, T)
    dim = size(E.X, 1)
    n = length(corpus)
    C = zeros(Float32, dim, n)
    # Threads.@threads
    for i in 1:n
        vectorize_as_prototype!(view(C, :, i), corpus[i], E, T)
    end
    
    C
end

function create_index(db, recall)
    dist = NormalizedCosineDistance()
    index = SearchGraph(; dist, db, verbose=false)
    index!(index; callbacks=SearchGraphCallbacks(MinRecall(recall)))
    optimize!(index, MinRecall(recall))
    index
end

create_index (generic function with 1 method)

In [5]:
@time E = embeddings()

 34.632897 seconds (11.18 M allocations: 11.654 GiB, 2.34% gc time, 1.33% compilation time)


(X = Float32[-0.039261177 -0.044748805 … -0.051247735 0.05427054; 0.09036921 0.03974378 … -0.017566958 0.06556069; … ; -0.043778244 0.04899757 … -0.038878556 -0.097423464; 0.021343287 0.012799488 … -0.0017221138 0.006681864], vocab = ["</s>", "_usr", "que", "de", ",", ".", "y", "a", "la", "no"  …  "tannehil", "movelike", "peliblanca", "cristianzav", "rqq", "preguntartee", "sutedes", "venciendolo", "eviscerado", "fectivo"])

In [6]:
display(@htl "<h1>Cargando el corpus</h1>")

include("read_datasets.jl")
D, Q = read_news()
T = text_model_and_vectors(D.corpus, E.vocab)
@show unique(D.labels), T.model

<h1>Cargando el corpus</h1>

(unique(D.labels), T.model) = (["AdriDelgadoRuiz", "El_Universal_Mx", "CNNEE", "NTN24", "UniNoticias", "TelemundoNews", "SinEmbargoMX", "Reforma", "abc_es", "azucenau", "AristeguiOnline", "el_pais", "EFEnoticias", "PublimetroMX", "PublimetroChile", "Radio_Formula", "RicardoAlemanMx", "epigmenioibarra", "Milenio", "LaRazon_mx", "abrahamendieta", "PublimetroCol", "teleSURtv", "bbcmundo", "julioastillero"], {VectorModel global_weighting=BinaryGlobalWeighting(), local_weighting=TpWeighting(), train-voc=438136, train-n=30244, maxoccs=43239})


(["AdriDelgadoRuiz", "El_Universal_Mx", "CNNEE", "NTN24", "UniNoticias", "TelemundoNews", "SinEmbargoMX", "Reforma", "abc_es", "azucenau"  …  "Radio_Formula", "RicardoAlemanMx", "epigmenioibarra", "Milenio", "LaRazon_mx", "abrahamendieta", "PublimetroCol", "teleSURtv", "bbcmundo", "julioastillero"], {VectorModel global_weighting=BinaryGlobalWeighting(), local_weighting=TpWeighting(), train-voc=438136, train-n=30244, maxoccs=43239})

In [7]:
@time C = vectorize_corpus_as_prototypes(D.corpus, E, T)

  1.762749 seconds (3.32 M allocations: 311.708 MiB, 7.56% gc time, 26.48% compilation time)


300×30244 Matrix{Float32}:
 -0.0423962   -0.0251833    -0.0519478    …  -0.0799066   -0.10502
  0.0103375   -0.00889515   -0.00474732       0.0161733    0.0393123
  0.0049359    0.00402093   -0.00378763      -0.00500099   0.0214965
 -0.0453966    0.00096476   -0.00438061       0.0346591    0.0259731
  0.0604199    0.0177876     0.00936113      -0.0200484   -0.00529611
  0.0789069   -0.00163282    0.036198     …   0.0155074    0.0160663
 -0.0572526   -0.000435368   0.0127748       -0.00196163  -0.00115164
  0.0680706   -0.000611415  -0.010575        -0.038946     0.0171894
  0.0752703    0.032632      0.0883861        0.0999684    0.120996
  0.0161864   -0.0126705     2.09727f-5      -0.0145413   -0.0141206
  0.0662018    0.0401931     0.0825404    …   0.0804474    0.0961554
  0.0830081   -0.0211806     0.032581         0.0237437    0.0534497
  0.073465    -0.0110519     0.0609841        0.0552063    0.0185002
  ⋮                                       ⋱               
 -0.0227881   -0.0

### Se crea el índice métrico

In [8]:
@time index = create_index(MatrixDatabase(C), 0.95)

  6.825807 seconds (12.93 M allocations: 1.448 GiB, 4.16% gc time, 85.67% compilation time)


SearchGraph{NormalizedCosineDistance, MatrixDatabase{Matrix{Float32}}, BeamSearch}
  dist: NormalizedCosineDistance NormalizedCosineDistance()
  db: MatrixDatabase{Matrix{Float32}}
  links: Array{Vector{Int32}}((30244,))
  locks: Array{Base.Threads.SpinLock}((30244,))
  hints: Array{Int32}((102,)) Int32[1, 48, 50, 52, 154, 222, 229, 235, 259, 308  …  2102, 2126, 2148, 2155, 2170, 2261, 2265, 2291, 2293, 2298]
  search_algo: BeamSearch
  verbose: Bool false


### Búsqueda de todos los vecinos cercanos en el vocabulario, observe la conveniencia del uso de un índice

Entre más sean las consultas más se ve la bondad (~42k consultas)

In [9]:
let k = 11
    Qvectors = MatrixDatabase(vectorize_corpus_as_prototypes(Q.corpus, E, T))
    t1 = @elapsed I, _ = searchbatch(index, Qvectors, k)
    # WARNING: Don't run the following line, it takes too much time
    ex = ExhaustiveSearch(; db=index.db, dist=index.dist)
    t2 = @elapsed gI, _ = searchbatch(ex, Qvectors, k)
    r = macrorecall(gI, I)
    n = size(I, 2)
    labels = String.(D.labels)
    s1 = scores(Q.labels, knn(I, labels))
    s2 = scores(Q.labels, knn(gI, labels))
    
    @htl """
    <div>(brute vs indexed) macro-recall: $r, n: $n</div>
    <div>searchgraph search time: $t1, scores: $s1</div>
    <div>brute force search time: $t2, scores: $s2</div>
    """
end

<div>(brute vs indexed) macro-recall: 0.9367341970733968, n: 3361</div>
<div>searchgraph search time: 1.051562974, scores: <span class="Core-NamedTuple">(macrof1 = 0.23143995250110957, macrorecall = 0.2379487085465786, accuracy = 0.26331448973519783)</span></div>
<div>brute force search time: 0.593575376, scores: <span class="Core-NamedTuple">(macrof1 = 0.24237709344732106, macrorecall = 0.2483987112746582, accuracy = 0.27432311811960725)</span></div>

### Búsqueda y presentación de los resultados

In [10]:
function search_and_display(index, qtext, k, D, E, T)
    res = KnnResult(k)
    q = zeros(Float32, size(E.X, 1))
    vectorize_as_prototype!(q, qtext, E, T)
    @time search(index, q, res)
    
    L = []
    for (j, (id, d)) in enumerate(res)
        push!(L, @htl "<tr><td>$j</td><td>$id</td><td>$(round(d, digits=3))</td> <td>$(D.labels[id])</td><td> $(D.corpus[id])</td> </tr>")
    end

    display(@htl """<h2>resultados for "$qtext"</h2>
    <table>
    <th>  <td>id</td> <td>dist</td> <td>user</td> </td>message<td> </th>
        $L
    </table>
    """)
end


search_and_display (generic function with 1 method)

In [11]:

display(@htl "<h1>Ejemplos de búsqueda</h1>")
search_and_display(index, "el gobierno de andres manuel lopez", 7, D, E, T)
search_and_display(index, "trafico de drogas", 7, D, E, T)
search_and_display(index, "covid corona virus", 7, D, E, T)


<h1>Ejemplos de búsqueda</h1>

  0.000167 seconds


1,5559,0.113,Radio_Formula,El presidente Andrés Manuel López Obrador exhibió el supuesto sueldo de Loret de Mola... con un documento con errores ortográficos. https://t.co/nx4bDFfw8p
2,3647,0.122,CNNEE,Así pensó un plan B el presidente de México Andrés Manuel López Obrador para organizar la consulta de revocación de mandato. https://t.co/GaAeHdjhAG
3,2292,0.124,El_Universal_Mx,RT @ElUniversal_SLP: #Entérate El gobernador Ricardo Gallardo Cardona pidió al presidente Andrés Manuel López Obrador apoyo para reforzar l…
4,383,0.124,azucenau,Informa @RicardoAnayaC que ha sido citado en el Reclusorio Norte por lo que acusa persecución de parte del presidente Andrés Manuel López Obrador
5,27877,0.127,azucenau,"#ALMOMENTO | Por segundo día consecutivo, el presidente Andrés Manuel López Obrador salió esta tarde de Palacio Nacional. https://t.co/tdBqBYhhuc"
6,24904,0.135,azucenau,"#AzucenaxMilenio | Durante #LaMañanera, el presidente López Obrador respaldó nuevamente al gobernador de Veracruz, Cuitláhuac García https://t.co/iX2ubBHbyJ"
7,1231,0.136,azucenau,"#ENVIVO | Mensaje conjunto del presidente López Obrador y Alejandro Giammattei, presidente de #Guatemala https://t.co/uOwcCHGKt3"


  0.000815 seconds


1,22544,0.245,LaRazon_mx,#SSPC: Red de tráfico de migrantes es ubicada en #Guanajuato y #Tamaulipas https://t.co/2TdiV3G2Zf https://t.co/hctIUWIx7G
2,22011,0.263,AristeguiOnline,El comercio ilegal de drogas se digitaliza con la pandemia https://t.co/dzZtvcjBAA https://t.co/hrG2x5a5xs
3,13880,0.277,Radio_Formula,"#ÚltimaHora 🚨 Emma Coronel, esposa del ""Chapo"" Guzmán, se declara culpable de tráfico de drogas y lavado de dinero https://t.co/25f8JceKUw https://t.co/p1UHTs3Ln9"
4,7925,0.283,PublimetroMX,"Ricardo y Yolanda Salazar, familiares de la primera esposa del Chapo Guzmán, están detenidos por tráfico de drogas. https://t.co/XysLhWruIf"
5,3848,0.285,Reforma,"El 6 de enero de 2021 inició el juicio de Joaquín Naasón García Los cargos estaban relacionados con el abuso de menores, producción de pronografía e incluso tráfico de personas, entre otros. https://t.co/8qwEvydrf4"
6,9124,0.285,EFEnoticias,"Un centenar de profesores mantienen cortado esta mañana el tráfico en la Ronda Litoral, a la altura de la Vila Olímpica de Barcelona, lo que está causando grandes retenciones de tráfico en esta vía en el tercer día de huelga de la enseñanza. https://t.co/jIsziPxkn7"
7,16919,0.286,LaRazon_mx,. @ONU_es aprueba resolución mexicana para combatir tráfico de armas https://t.co/B7YEbjID5n https://t.co/WBD30vdeJa


  0.002049 seconds (2 allocations: 14.375 KiB)


1,28803,0.452,bbcmundo,Los vacunados transmiten menos el virus. https://t.co/BYtDcmYqwj
2,7976,0.462,abc_es,Primeros datos de la eficacia de dos antivirales frente al virus de la viruela del simio https://t.co/TdBHvIIT3w
3,3632,0.468,AristeguiOnline,Molnupiravir elimina el virus SARS-CoV-2 activamente infeccioso al tercer día de comenzar la terapia: estudio https://t.co/93yBkWlj3G
4,6122,0.476,El_Universal_Mx,Es el primer deceso registrado en la entidad por esa variante del virus SARS-CoV-2 https://t.co/3FlnzSNpbk
5,15380,0.477,bbcmundo,El alentador efecto de la vacuna contra el virus del papiloma humano (VPH) en la reducción del cáncer de cuello uterino https://t.co/rQV9L9Q9F4
6,14629,0.482,SinEmbargoMX,Fármacos contra enfermedad inflamatoria intestinal protegerían de la COVID grave https://t.co/b7yXzIGahI https://t.co/nGZVBpX9Xi
7,27059,0.485,AristeguiOnline,#Video | Se corona Barbora Krejcikova en Roland Garros https://t.co/4TLdKn4pVU https://t.co/8QSjnu0I3r


In [12]:
display(@htl "<h1>Ejemplos de búsqueda (mensajes aleatorios)</h1>")

for i in 1:3
    for qid in rand(1:length(D.corpus))
        search_and_display(index, D.corpus[qid], 7, D, E, T)
    end
end

<h1>Ejemplos de búsqueda (mensajes aleatorios)</h1>

  0.000385 seconds


1,18790,0.0,CNNEE,"ANÁLISIS Mientras occidente teme un ataque de Rusia a Ucrania, los medios estatales rusos lo pintan como un plan de la OTAN para tomar el control de su energía. https://t.co/pHWO4RA5Ui"
2,7116,0.055,EFEnoticias,"El presidente de Ucrania, Volodímir Zelenski, pidió este sábado a los ucranianos que no se dejen llevar por el pánico ante el drástico incremento de las informaciones sobre la inminencia de un ataque ruso. https://t.co/OR0AVaBSqn"
3,5841,0.056,bbcmundo,"Edward Fishman, exespecialista del gobierno de EE.UU., evalúa en una entrevista con @GerardoLissardy el efecto de las sanciones económicas a Rusia por invadir a Ucrania y advierte que el objetivo ya no es cambiar el comportamiento de Vladimir Putin. https://t.co/IDYPKCsWBa"
4,15811,0.057,CNNEE,"Los líderes de Europa acordaron la eliminación gradual de las importaciones de carbón ruso como parte de un nuevo paquete de sanciones impulsadas por la evidencia de actos atroces en Bucha, Ucrania. Un embargo al petróleo podría ser lo siguiente. https://t.co/ZxYZzHzbny"
5,10500,0.058,NTN24,"""Puede darse un escenario en el que Rusia busque lanzar una ofensiva de guerra no militar, económica y cibernética contra los adversarios de Venezuela que serían principalmente Colombia y Guyana"": Joseph Humire, experto en seguridad global https://t.co/CXyaotioAD https://t.co/7bublWGlvB"
6,18374,0.058,el_pais,"En este centro conviven ucranios, rusos o bielorrusos. El régimen de Putin no les representa: unas han escapado de la homofobia del régimen ruso y otros, de la invasión de su país. “No es un problema entre rusos y ucranios, el problema es Putin” https://t.co/NU8E26pegp"
7,688,0.058,CNNEE,"La secretaria del Tesoro de EE.UU. admitió que falló en anticipar el tiempo que afectaría la alta inflación a los consumidores estadounidenses, mientras el gobierno de Biden trabaja para contener un creciente lastre político. https://t.co/ZyR78dmdcC"


  0.000608 seconds


1,26571,0.0,epigmenioibarra,RT @rturrent: Recuerdo cómo soñaron con cancelar #DosBocas o con que no se terminara a tiempo. Hoy se inaugura la refinería que tiene acer…
2,24437,0.073,el_pais,"Este fin de semana se cambia la hora, pero eso no va a ayudar a la conciliación. La clave es que España no está en el huso horario que le corresponde, lo que se traduce en que los españoles llevan dos horas de retraso con el resto de Europa #hemeroteca https://t.co/gkVNV1jE2V"
3,24612,0.073,Milenio,"▶ ¿Qué tanto ocurre con el avión presidencial? Aunque lo propusieron para llevar a los atletas olímpicos a #Tokio2021, salieron voces en contra diciendo que no es viable ¿Qué se sabe al respecto? ¿Lo usarán o no? 📺 La historia con @azucenau #AzucenaxMILENIO https://t.co/KeoXJbuIIS"
4,1505,0.075,el_pais,"RT @el_pais_madrid: “¿Cómo se va a pagar el alquiler si se está diciendo que no se use?”, se queja una fuente de la confianza del president…"
5,25940,0.076,bbcmundo,"“En México no se puede vivir tranquilo, porque cada vez que tu hijo sale, estás con el pendiente de si va a regresar bien”. Mario Escobar, padre de la joven Debanhi que apareció sin vida la pasada semana y cuya muerte aún se investiga, habló con BBC Mundo. https://t.co/BKFJtwx5Ha"
6,12275,0.076,AdriDelgadoRuiz,"""El problema es que la gente se conduce como si ya no hubiera pandemia. Basta salir un minuto a la calle para darse cuenta que se saturan lugares y que no se usa el cubreboca"". Dijo el Dr. Antonio Lazcano para #ElDedoenlaLLaga. #ElHeraldoRadio del @heraldodemexico."
7,21507,0.076,epigmenioibarra,"“Transformación es un eufemismo -me dijo AMLO- aquí lo que está en marcha es una revolución”.Una revolución como no ha habido otra en la historia, que se produce sin violencia y en libertad, en la que no se controla ni se reprime a nadie. Pido ver y dar RT https://t.co/Kl5W8yfrS4"


  0.000327 seconds


1,17349,0.0,azucenau,#AzucenaALas10 | La #CNDH acreditó que la fiscalía de #Veracruz detuvo arbitrariamente a José Manuel del Río Virgen El gobernador Cuitláhuac García aseguró que no hubo violación de sus derechos https://t.co/5V03Kn6cPC
2,12415,0.062,Milenio,"▶️ Mientras AMLO respalda a Cuitláhuac Garcia, gobernador de Veracruz, en su estado siguen las denuncias de abuso de autoridad Andrea está presa porque no quiso declarar contra un sujeto ya que ella no sabía nada del caso; así la justicia 📺 La historia con @azucenau https://t.co/1K2vPbmLzL"
3,29415,0.065,Milenio,"#Trascendió | Que la #FGR niega a Lourdes Mendoza una copia del acta certificada de la denuncia de Emilio Lozoya en su contra, pese a que en la mañanera del 20 de agosto el Presidente planteó que se hiciera pública https://t.co/q8zk82ylFu https://t.co/jQjJq5ItxE"
4,2467,0.067,RicardoAlemanMx,"Esto luego de que el gobierno de Cuitláhuac García detuvo al secretario Técnico de la Junta de Coordinación Política (Jucopo) del Senado, José Manuel del Río Virgen. https://t.co/P6PeYpvxxd"
5,5106,0.068,CNNEE,ÚLTIMA HORA El Poder Judicial de Honduras informó este miércoles en sus canales oficiales que el juez de primera instancia otorgó la solicitud de extradición que involucra a el expresidente Juan Orlando Hernández Alvarado. https://t.co/fCNDWhZLiy
6,28665,0.069,Milenio,"▶️ Alejandro Encinas, subsecretario de Gobernación, acusó al Poder Judicial de entorpecer investigación sobre Ayotzinapa Señaló que un juez eximió a dos hermanos presuntos miembros de 'Los Tilos', ¿qué sucedió con el caso? 📺 La historia con @AlexDominguezB #AlexEnMILENIO https://t.co/dy3bdhg3oT"
7,19321,0.07,SinEmbargoMX,"🚨#ÚltimaHora | Alfredo Ramírez denunció que la Auditoría Superior del estado investiga el presunto fraude por 5,186 millones de pesos en la construcción, el insólito arrendamiento y la compra final de los cuarteles regionales de la policía estatal https://t.co/pMfce4Kzq7 https://t.co/l246JEOxGe"


# Actividades
- Reproduzca el ejercicio de este notebook, use embeddings para español, cambié los ejemplos. Se sugiere el uso de <https://ingeotec.github.io/regional-spanish-models/> donde encontrará modelos fastText regionalizados del español, pero puede usar otros embeddings.
- ¿Qué piensa de las diferencias de tamaño entre los documentos y las consultas? esto como afecta a la representación semántica.
- ¿Cuál sería el símil de bigramas y trigramas para este esquema de representación semántica? Implementelo.
- Implemente su búsqueda semántica. Si usa Julia considere `SimilaritySearch.jl` y si usa Python considere `faiss`.
- Reporte su notebook y anote sus soluciones a las preguntas planteadas. Reporte los resultados de sus implementaciones, compare contra las alternativas presentadas en este reporte. Discuta sus resultados. Finalice el reporte  con reflexiones sobre el uso de nubes de puntos en lugar de bolsas de palabras tradicionales. Anoté sus conclusiones.

# Bibliografía
- [KSKW2015] Kusner, M., Sun, Y., Kolkin, N., & Weinberger, K. (2015, June). From word embeddings to document distances. In International conference on machine learning (pp. 957-966). PMLR.
- [PW2009] Pele, O., & Werman, M. (2009, September). Fast and robust earth mover's distances. In 2009 IEEE 12th international conference on computer vision (pp. 460-467). IEEE.
